In [67]:
# основные библиотеки для работы с данными
import pandas as pd
import numpy as np
import re

# инструменты визуализации
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [68]:
pd.set_option('display.max_columns', 30)

In [69]:
# загружаем датасет, который был частично обработан на этапе первого релиза
# логику обработки и принятия решений можно посмотреть в:
# realises/Release_1/EDA_part_1.ipynb
# Здесь работа с датасетом продолжается на том месте, на котором остановились
# на предыдущем этапе
# Датасеты "слили" в общий опять, и работали уже над одни объемом
df = pd.read_csv('/Users/alex_eyfler/ds_bootcamp/rent_real_estate_team_pandas/Alexander/data/full_cleaned_data.csv')

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23199 entries, 0 to 23198
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID  объявления       23199 non-null  int64  
 1   Количество комнат    22170 non-null  float64
 2   Метро                21893 non-null  object 
 3   Парковка             9871 non-null   object 
 4   Ремонт               20454 non-null  object 
 5   Балкон               15287 non-null  float64
 6   Окна                 16643 non-null  object 
 7   Высота потолков, м   11122 non-null  float64
 8   Мусоропровод         12743 non-null  object 
 9   Общая площадь, м2    23199 non-null  float64
 10  Тип дома             23199 non-null  object 
 11  Этажность дома       23199 non-null  int64  
 12  Этаж квартиры        23199 non-null  int64  
 13  Лоджия               15287 non-null  float64
 14  Санузел совмещенный  20559 non-null  float64
 15  Санузел раздельный   20559 non-null 

In [71]:
df.isnull().sum()

ID  объявления             0
Количество комнат       1029
Метро                   1306
Парковка               13328
Ремонт                  2745
Балкон                  7912
Окна                    6556
Высота потолков, м     12077
Мусоропровод           10456
Общая площадь, м2          0
Тип дома                   0
Этажность дома             0
Этаж квартиры              0
Лоджия                  7912
Санузел совмещенный     2640
Санузел раздельный      2640
Лифт пасс               5469
Лифт груз               5469
Можно с детьми          6287
Можно с животными      16102
Цена аренды                0
dtype: int64

In [72]:
# делали глубокую копию датафрейма, чтобы работать уже с ней
# планировалось поменять переменные и причесать все в красивый вид,
# но времени не хватило
df_tested = df.copy()

In [73]:
# если не указано в явном виде в объявлении, что можно с животными
# принимаем то, что с животными нельзя и заполняем колонку нулями

df_tested['Можно с животными'] = df_tested['Можно с животными'].fillna(0)

In [74]:
# по такой же логике коллегиально приняли решение заполнять пропуски, если нет
# явного указания в объявлении на то, что можно с детьми

df_tested['Можно с детьми'] = df_tested['Можно с детьми'].fillna(0)

In [75]:
# заполняем пропуски в колонках с лифтами
# принято решение, что если этажность дома больше 5, то добавляем по одному
# пассажирскому и одному грузовому лифту

df_tested.loc[
    (df_tested['Этажность дома'] > 5) & 
    (df_tested['Лифт пасс'].isna()) &
    (df_tested['Лифт груз'].isna()),
    ['Лифт пасс', 'Лифт груз']
    ] = 1

In [76]:
# оставшиеся пропуски заполняем нулями, т.к. если этажность дома меньше либо равна 5,
# принято решение, что лифтов в таком доме лифтов нет

df_tested[['Лифт пасс', 'Лифт груз']] = df_tested[['Лифт пасс', 'Лифт груз']].fillna(0)

In [77]:
# заполняем пропущенное значение санузла
# если не указан никакой из санузлов в объявлении,
# значит считаем, что один совмещенный

df_tested.loc[
    (df_tested['Санузел совмещенный'].isna()) &
    (df_tested['Санузел раздельный'].isna()),
    ['Санузел совмещенный']
    ] = 1

In [78]:
# заполняем оставшиеся пропуски нулевыми значениями
df_tested['Санузел совмещенный'] = df_tested['Санузел совмещенный'].fillna(0)
df_tested['Санузел раздельный'] = df_tested['Санузел раздельный'].fillna(0)

In [79]:
# если в изначальном объявлении не указано ни о лоджии, ни о балконе,
# считаем, что их нет и зануляем значение
# т.к. это конкурентное преимущество, и было бы явно указано о наличии балкона
# и лоджии, если бы они были

df_tested['Лоджия'] = df_tested['Лоджия'].fillna(0)
df_tested['Балкон'] = df_tested['Балкон'].fillna(0)

In [80]:
# если мусоропровод не указан, принимаем, что его нет
df_tested['Мусоропровод'] = df_tested['Мусоропровод'].fillna('Нет')

In [81]:
# заполняем пропуски по высоте потолков
# используем медиану, игнорируя NaN значения при ее вычислении
# медиана принята как нечувствительная к выбросам величина

median_value = np.nanmedian(df_tested['Высота потолков, м'])
df_tested['Высота потолков, м'] = df_tested['Высота потолков, м'].fillna(median_value)

In [82]:
# заполняем пропущенные значения по наиболее часто встречающейся величине - моде
# моду смотрели в процессе работы через value_counts()

df_tested['Окна'] = df_tested['Окна'].fillna('Во двор')

In [83]:
# аналогично, по моде заполняем пропущенные значения ремонта

df_tested['Ремонт'] = df_tested['Ремонт'].fillna('Евроремонт')

In [84]:
# аналогично, по моде заполняем пропущенные значения для парковки
df_tested['Парковка'] = df_tested['Парковка'].fillna('наземная')

In [85]:
# заполнить пропуски метро по моде оказалось некорректно, т.к. небольшая разница
# между самым частым значением и последующими
# принято решение распределить по топ5 самым частым значениям

top_5_stations = df_tested['Метро'].value_counts().index[:5]

def fill_with_top_5(row, top_5):
    if pd.isna(row['Метро']):
        return np.random.choice(top_5)  # Случайный выбор станции из топ-5
    return row['Метро']


df_tested['Метро'] = df_tested.apply(lambda row: fill_with_top_5(row, top_5_stations), axis=1)


In [86]:
# количество комнат тоже нельзя однозначно описать по моде
# поэтому написали логику замены значений в зависимости от значений общей площади
# соотношение площади и количества комнат выявили собственным эмпирическим путем :)

df_tested.loc[(df_tested['Количество комнат'].isna()) & (df_tested['Общая площадь, м2'] < 40), 'Количество комнат'] = 1
df_tested.loc[(df_tested['Количество комнат'].isna()) & (df_tested['Общая площадь, м2'] >= 40) & (df_tested['Общая площадь, м2'] < 60), 'Количество комнат'] = 2
df_tested.loc[(df_tested['Количество комнат'].isna()) & (df_tested['Общая площадь, м2'] >= 60) & (df_tested['Общая площадь, м2'] < 80), 'Количество комнат'] = 3
df_tested.loc[(df_tested['Количество комнат'].isna()) & (df_tested['Общая площадь, м2'] >= 80) & (df_tested['Общая площадь, м2'] < 100), 'Количество комнат'] = 4
df_tested.loc[(df_tested['Количество комнат'].isna()) & (df_tested['Общая площадь, м2'] >= 100), 'Количество комнат'] = 5


In [87]:
# удалим полные дубликаты по всем колонкам, за исключением ID и цены аренды
# чтобы с одинаковыми наборами фичей у нас не получились разные цены и модель
# "не запуталась", как мы предположили

df_tested = df_tested.drop_duplicates(subset=[
    'Количество комнат',
    'Метро',
    'Парковка',
    'Ремонт',
    'Балкон',
    'Окна',
    'Высота потолков, м',
    'Мусоропровод',
    'Общая площадь, м2',
    'Тип дома',
    'Этажность дома',
    'Этаж квартиры',
    'Лоджия',
    'Санузел совмещенный',
    'Санузел раздельный',
    'Лифт пасс',
    'Лифт груз',
    'Можно с детьми',
    'Можно с животными'
    ]
                          )

In [88]:
# сохранение файла после препроцессинга для дальнейших манипуляций по преобразованию типов данных колонок
# df_tested.to_csv('/Users/alex_eyfler/ds_bootcamp/rent_real_estate_team_pandas/Alexander/data/almost_final_full_data.csv', index=False)

In [89]:
# Переименовываем столбцы

df_tested.rename(columns={
    'ID  объявления': 'id',
    'Количество комнат': 'number_of_rooms',
    'Метро': 'subway',
    'Парковка': 'parking',
    'Ремонт': 'renovation',
    'Балкон': 'balcony',
    'Окна': 'windows',
    'Высота потолков, м': 'ceiling_height_m',
    'Мусоропровод': 'garbage_chute',
    'Общая площадь, м2': 'total_area_m2',
    'Тип дома': 'building_type',
    'Этажность дома': 'building_floors',
    'Этаж квартиры': 'apartment_floor',
    'Лоджия': 'loggia',
    'Санузел совмещенный': 'combined_bathroom',
    'Санузел раздельный': 'separate_bathroom',
    'Лифт пасс': 'passenger_elevator',
    'Лифт груз': 'freight_elevator',
    'Можно с детьми': 'allowed_with_children',
    'Можно с животными': 'allowed_with_pets',
    'Цена аренды': 'rental_price'
}, inplace=True)


In [90]:
# удалим столбец subway
df_tested.drop(labels='subway', axis=1, inplace=True)

Все значения в категориальных данных будут транслитерированы либо переведены на английский

In [91]:
# # metro_translation = {
#     'Смоленская': 'Smolenskaya',
#     'Арбатская': 'Arbatskaya',
#     'Кропоткинская': 'Kropotkinskaya',
#     'Александровский сад': 'Alexandrovsky_Sad',
#     'Киевская': 'Kievskaya',
#     'Библиотека им': 'Biblioteka_Imeni_Lenina',
#     'Боровицкая': 'Borovitskaya',
#     'Баррикадная': 'Barrikadnaya',
#     'Краснопресненская': 'Krasnopresnenskaya',
#     'Селигерская': 'Seligerskaya',
#     'Марк': 'Mark',
#     'Алтуфьево': 'Altufyevo',
#     'Новодачная': 'Novodachnaya',
#     'Долгопрудная': 'Dolgoprudnaya',
#     'Ховрино': 'Khovrino',
#     'Бабушкинская': 'Babushkinskaya',
#     'Лианозово': 'Lianozovo',
#     'Новогиреево': 'Novogireevo',
#     'Щелковская': 'Shchyolkovskaya',
#     'Достоевская': 'Dostoevskaya',
#     'Кузнецкий мост': 'Kuznetsky_Most',
#     'Сухаревская': 'Sukharevskaya',
#     'Проспект Мира': 'Prospekt_Mira',
#     'Цветной бульвар': 'Tsvetnoy_Bulvar',
#     'Трубная': 'Trubnaya',
#     'Комсомольская': 'Komsomolskaya',
#     'Рижская': 'Rizhskaya',
#     'Сретенский бульвар': 'Sretensky_Bulvar',
#     'Тургеневская': 'Turgenevskaya',
#     'Марьина роща': 'Maryina_Roshcha',
#     'Коломенская': 'Kolomenskaya',
#     'Кожуховская': 'Kozhukhovskaya',
#     'Депо': 'Depo',
#     'Технопарк': 'Tekhnopark',
#     'Ленинский проспект': 'Leninsky_Prospekt',
#     'Автозаводская': 'Avtozavodskaya',
#     'Пражская': 'Prague',
#     'Алма-Атинская': 'Alma_Atinskaya',
#     'Улица Академика Янгеля': 'Ulitsa_Akademika_Yangelya',
#     'Зябликово': 'Zyablikovo',
#     'Царицыно': 'Tsaritsyno',
#     'Тульская': 'Tulskaya',
#     'Шипиловская': 'Shipilovskaya',
#     'Кантемировская': 'Kantemirovskaya',
#     'Орехово': 'Orekhovo',
#     'Дубровка': 'Dubrovka',
#     'ЗИЛ': 'ZIL',
#     'Чертановская': 'Chertanovskaya',
#     'Каховская': 'Kakhovskaya',
#     'Павелецкая': 'Paveletskaya',
#     'Нагорная': 'Nagornaya',
#     'Нахимовский проспект': 'Nakhimovsky_Prospekt',
#     'Каширская': 'Kashirskaya',
#     'Крымская': 'Krymskaya',
#     'Верхние котлы': 'Verkhnie_Kotly',
#     'Красногвардейская': 'Krasnogvardeyskaya',
#     'Домодедовская': 'Domodedovskaya',
#     'Лесопарковая': 'Lesoparkovaya',
#     'Севастопольская': 'Sevastopolskaya',
#     'Покровское': 'Pokrovskoye',
#     'Шаболовская': 'Shabolovskaya',
#     'Аннино': 'Annino',
#     'Варшавская': 'Varshavskaya',
#     'Нагатинская': 'Nagatinskaya',
#     'Крестьянская застава': 'Krestyanskaya_Zastava',
#     'Пролетарская': 'Proletarskaya',
#     'Площадь Гагарина': 'Ploshchad_Gagarina',
#     'Битца': 'Bitsa',
#     'Южная': 'Yuzhnaya',
#     'Москворечье': 'Moskvorechye',
#     'Новочеркасская': 'Novocherkasskaya',
#     'Ладожская': 'Ladozhskaya',
#     'Лесная': 'Lesnaya',
#     'Площадь Ленина': 'Ploshchad_Lenina',
#     'Гражданский проспект': 'Grazhdansky_Prospekt',
#     'Академическая': 'Akademicheskaya',
#     'Девяткино': 'Devyatkino',
#     'Проспект Большевиков': 'Prospekt_Bolshevikov',
#     'Площадь Мужества': 'Ploshchad_Muzhestva',
#     'Зенит': 'Zenit',
#     'Площадь Александра Невского': 'Ploshchad_Alexandra_Nevskogo',
#     'Московская': 'Moskovskaya',
#     'Купчино': 'Kupchino',
#     'Фрунзенская': 'Frunzenskaya',
#     'Звездная': 'Zvezdnaya',
#     'Парк Победы': 'Park_Pobedy',
#     'Электросила': 'Elektrosila',
#     'Московские ворота': 'Moskovskie_Vorota',
#     'Проспект Славы': 'Prospekt_Slavy',
#     'Митино': 'Mitino',
#     'Волоколамская': 'Volokolamskaya',
#     'Пятницкое шоссе': 'Pyatnitskoye_Shosse',
#     'Пенягино': 'Penyagino',
#     'Люблино': 'Lyublino',
#     'Братиславская': 'Bratislavskaya',
#     'Перерва': 'Pererva',
#     'Волжская': 'Volzhskaya',
#     'Кубанская': 'Kubanskaya',
#     'Котельники': 'Kotelniki',
#     'Марьино': 'Maryino',
#     'Печатники': 'Pechatniki',
#     'Планерная': 'Planernaya',
#     'Речной вокзал': 'Rechnoy_Vokzal',
#     'Сходненская': 'Skhodnenskaya',
#     'Кузьминки': 'Kuzminki',
#     'Окская': 'Okskaya',
#     'Рязанский проспект': 'Ryazansky_Prospekt',
#     'Текстильщики': 'Tekstilshchiki',
#     'Коптево': 'Koptevo',
#     'Красный Балтиец': 'Krasny_Baltiets',
#     'Войковская': 'Voykovskaya',
#     'Окружная': 'Okruzhnaya',
#     'Петровско-Разумовская': 'Petrovsko_Razumovskaya',
#     'Лихоборы': 'Likhobory',
#     'Верхние Лихоборы': 'Verkhnie_Likhobory',
#     'Сокол': 'Sokol',
#     'Водный стадион': 'Vodny_Stadion',
#     'Борисово': 'Borisovo',
#     'Отрадное': 'Otradnoye',
#     'Владыкино': 'Vladykino',
#     'Бибирево': 'Bibirevo',
#     'Дегунино': 'Degunino',
#     'Строгино': 'Strogino',
#     'Мякинино': 'Myakinino',
#     'Щукинская': 'Shchukinskaya',
#     'Тушинская': 'Tushinskaya',
#     'Беломорская': 'Belomorskaya',
#     'Ботанический сад': 'Botanichesky_Sad',
#     'ВДНХ': 'VDNKh',
#     'Ростокино': 'Rostokino',
#     'Выставочный центр': 'Vystavochny_Tsentr',
#     'Улица Сергея Эйзенштейна': 'Ulitsa_Sergeya_Eisensteina',
#     'Серпуховская': 'Serpukhovskaya',
#     'Третьяковская': 'Tretyakovskaya',
#     'Новокузнецкая': 'Novokuznetskaya',
#     'Добрынинская': 'Dobryninskaya',
#     'Таганская': 'Taganskaya',
#     'Мичуринский проспект': 'Michurinsky_Prospekt',
#     'Минская': 'Minskaya',
#     'Славянский бульвар': 'Slavyansky_Bulvar',
#     'Озерная': 'Ozyornaya',
#     'Давыдково': 'Davydkovo',
#     'Аминьевская': 'Aminevskaya',
#     'Раменки': 'Ramenki',
#     'Проспект Вернадского': 'Prospekt_Vernadskogo',
#     'Локомотив': 'Lokomotiv',
#     'Первомайская': 'Pervomayskaya',
#     'Черкизовская': 'Cherkizovskaya',
#     'Измайловская': 'Izmailovskaya',
#     'Свиблово': 'Sviblovo',
#     'Медведково': 'Medvedkovo',
#     'Багратионовская': 'Bagrationovskaya',
#     'Ломоносовский проспект': 'Lomonosovsky_Prospekt',
#     'Юго-Западная': 'Yugo_Zapadnaya',
#     'Рабочий поселок': 'Rabochy_Posyolok',
#     'Калужская': 'Kaluzhskaya',
#     'Кунцевская': 'Kuntsevskaya',
#     'Кутузовская': 'Kutuzovskaya',
#     'Говорово': 'Govorovo',
#     'Молодежная': 'Molodezhnaya',
#     'Новые Черемушки': 'Novye_Cheryomushki',
#     'Сетунь': 'Setun',
#     'Выставочная': 'Vystavochnaya',
#     'Университет': 'Universitet',
#     'Крылатское': 'Krylatskoye',
#     'Шелепиха': 'Shelepiha',
#     'Студенческая': 'Studencheskaya',
#     'Фили': 'Fili',
#     'Тропарево': 'Troparevo',
#     'Филевский парк': 'Filevsky_Park',
#     'Новаторская': 'Novatorskaya',
#     'Новопеределкино': 'Novoperedelkino',
#     'Солнцево': 'Solntsevo',
#     'Семеновская': 'Semyonovskaya',
#     'Соколиная гора': 'Sokolinnaya_Gora',
#     'Электрозаводская': 'Elektrozavodskaya',
#     'Измайлово': 'Izmailovo',
#     'Шоссе Энтузиастов': 'Shosse_Entuziastov',
#     'Лефортово': 'Lefortovo',
#     'Партизанская': 'Partizanskaya',
#     'Панфиловская': 'Panfilovskaya',
#     'Стрешнево': 'Streshnevo',
#     'Октябрьское поле': 'Oktyabrskoye_Pole',
#     'Зорге': 'Zorge',
#     'Покровское-Стрешнево': 'Pokrovskoye_Streshnevo',
#     'Полежаевская': 'Polezhayevskaya',
#     'Аэропорт': 'Aeroport',
#     'Охотный ряд': 'Okhotny_Ryad',
#     'Международная': 'Mezhdunarodnaya',
#     'Деловой центр': 'Delovoy_Tsentr',
#     'Белорусская': 'Belorusskaya',
#     'Маяковская': 'Mayakovskaya',
#     'Чистые пруды': 'Chistye_Prudy',
#     'Октябрьская': 'Oktyabrskaya',
#     'Лубянка': 'Lubyanka',
#     'Новослободская': 'Novoslobodskaya',
#     'Тестовская': 'Testovskaya',
#     'Парк Культуры': 'Park_Kultury',
#     'Курская': 'Kurskaya',
#     'Пушкинская': 'Pushkinskaya',
#     'Тверская': 'Tverskaya',
#     'Спортивная': 'Sportivnaya',
#     'Лужники': 'Luzhniki',
#     'Китай-город': 'Kitay_Gorod',
#     'Красные ворота': 'Krasnye_Vorota',
#     'Полянка': 'Polyanka',
#     'Чеховская': 'Chekhovskaya',
#     'Улица 1905 года': 'Ulitsa_1905_Goda',
#     'Дмитровская': 'Dmitrovskaya',
#     'Менделеевская': 'Mendeleyevskaya',
#     'Алексеевская': 'Alekseyevskaya',
#     'Фонвизинская': 'Fonvizinskaya',
#     'Тимирязевская': 'Timiryazevskaya',
#     'Улица Академика Королева': 'Ulitsa_Akademika_Korolyova',
#     'Бутырская': 'Butyrskaya',
#     'Савеловская': 'Savyolovskaya',
#     'Улица Милашенкова': 'Ulitsa_Milashenkova',
#     'Выхино': 'Vykhino',
#     'Юго-Восточная': 'Yugo_Vostochnaya',
#     'Лермонтовский проспект': 'Lermontovsky_Prospekt',
#     'Жулебино': 'Zhulebino',
#     'Косино': 'Kosino',
#     'Белокаменная': 'Belokamennaya',
#     'Преображенская площадь': 'Preobrazhenskaya_Ploshchad',
#     'Бульвар Рокоссовского': 'Bulvar_Rokossovskogo',
#     'Римская': 'Rimskaya',
#     'Некрасовка': 'Nekrasovka',
#     'Бауманская': 'Baumanskaya',
#     'Угрешская': 'Ugreskaya',
#     'Авиамоторная': 'Aviamotornaya',
#     'Волгоградский проспект': 'Volgogradsky_Prospekt',
#     'Новохохловская': 'Novokhokhlovskaya',
#     'Нижегородская': 'Nizhegorodskaya',
#     'Стахановская': 'Stakhanovskaya',
#     'Курьяново': 'Kuryanovo',
#     'Площадь Ильича': 'Ploshchad_Ilyicha',
#     'Калитники': 'Kalitniki',
#     'Народное ополчение': 'Narodnoye_Opolchenie',
#     'улица Народного ополчения': 'Ulitsa_Narodnogo_Opolcheniya',
#     'Хорошево': 'Khoroshevo',
#     'Балтийская': 'Baltiyskaya',
#     'Телецентр': 'Teletsentr',
#     'Москва-Товарная': 'Moskva_Tovarnaya',
#     'Зюзино': 'Zyuzino',
#     'Автово': 'Avtovo',
#     'Проспект Ветеранов': 'Prospekt_Veteranov',
#     'Беговая': 'Begovaya',
#     'Чкаловская': 'Chkalovskaya',
#     'Красносельская': 'Krasnoselskaya',
#     'Яхромская': 'Yakhromskaya',
#     'Бескудниково': 'Beskudnikovo',
#     'Новоясеневская': 'Novoyasenevskaya',
#     'Ясенево': 'Yasenevo',
#     'Битцевский парк': 'Bitsevsky_Park',
#     'Теплый Стан': 'Tepliy_Stan',
#     'Беляево': 'Belyaevo',
#     'Коньково': 'Konkovo',
#     'Марксистская': 'Marksistskaya',
#     'Каланчёвская': 'Kalanchevskaya',
#     'Профсоюзная': 'Profsoyuznaya',
#     'Спартак': 'Spartak',
#     'Гражданская': 'Grazhdanskaya',
#     'Динамо': 'Dinamo',
#     'Петроградская': 'Petrogradskaya',
#     'Горьковская': 'Gorkovskaya',
#     'Крестовский остров': 'Krestovsky_Ostrov',
#     'Черная речка': 'Chyornaya_Rechka',
#     'Выборгская': 'Vyborgskaya',
#     'Политехническая': 'Polytechnicheskaya',
#     'Перово': 'Perovo',
#     'Лухмановская': 'Lukhmanovskaya',
#     'Сокольники': 'Sokolniki',
#     'Улица Дмитриевского': 'Ulitsa_Dmitriyevskogo',
#     'Новокосино': 'Novokosino',
#     'Андроновка': 'Andronovka',
#     'Рыбацкое': 'Rybatskoye',
#     'Шушары': 'Shushary',
#     'Обухово': 'Obukhovo',
#     'Щербинка': 'Shcherbinka',
#     'Улица Скобелевская': 'Ulitsa_Skobelevskaya',
#     'Улица Горчакова': 'Ulitsa_Gorchakova',
#     'Бунинская аллея': 'Buninskaya_Alleya',
#     'Бульвар Дмитрия Донского': 'Bulvar_Dmitriya Donskogo',
#     'Бульвар Адмирала Ушакова': 'Bulvar_Admirala Ushakova',
#     'Бутово': 'Butovo',
#     'Воронцовская': 'Vorontsovskaya',
#     'Воробьевы горы': 'Vorobyovy_Gory',
#     'Мневники': 'Mnyovniki',
#     'Сколково': 'Skolkovo',
#     'Немчиновка': 'Nemchinovka',
#     'ЦСКА': 'CSKA',
#     'Петровский Парк': 'Petrovsky_Park',
#     'Хорошевская': 'Khoroshevskaya',
#     'Кировский завод': 'Kirovsky_Zavod',
#     'Красный Строитель': 'Krasny_Stroitel',
#     'Боровское шоссе': 'Borovskoye_Shosse',
#     'Рассказовка': 'Rasskazovka',
#     'Нахабино': 'Nakhabino',
#     'Комендантский проспект': 'Komendantsky_Prospekt',
#     'Проспект Просвещения': 'Prospekt_Prosveshcheniya',
#     'Старая Деревня': 'Staraya_Derevnya',
#     'Удельная': 'Udelnaya',
#     'Пионерская': 'Pionerskaya',
#     'Озерки': 'Ozerki',
#     'Сенная площадь': 'Sennaya_Ploshchad',
#     'Адмиралтейская': 'Admiralteyskaya',
#     'Садовая': 'Sadovaya',
#     'Технологический институт': 'Tekhnologichesky_Institut',
#     'Нарвская': 'Narvskaya',
#     'Звенигородская': 'Zvenigorodskaya',
#     'Невский проспект': 'Nevsky_Prospekt',
#     'Василеостровская': 'Vasileostrovskaya',
#     'Трикотажная': 'Trikotazhnaya',
#     'Саларьево': 'Salaryevo',
#     'Румянцево': 'Rumyantsevo',
#     'Площадь Восстания': 'Ploshchad_Vosstaniya',
#     'Чернышевская': 'Chernyshevskaya',
#     'Владимирская': 'Vladimirskaya',
#     'Гостиный двор': 'Gostiny_Dvor',
#     'Обводный канал': 'Obvodny_Kanal',
#     'Лиговский проспект': 'Ligovsky_Prospekt',
#     'Улица Старокачаловская': 'Ulitsa_Starokachalovskaya',
#     'Ломоносовская': 'Lomonosovskaya',
#     'Улица Дыбенко': 'Ulitsa_Dybenko',
#     'Елизаровская': 'Yelizarovskaya',
#     'Парнас': 'Parnas',
#     'Приморская': 'Primorskaya',
#     'Театральная': 'Teatralnaya',
#     'Павшино': 'Pavshino',
#     'Бухарестская': 'Bukharestskaya',
#     'Дунайская': 'Dunayskaya',
#     'Волковская': 'Volkovskaya'
# }


In [92]:
# df_tested['subway'] = df_tested['subway'].map(metro_translation)

In [93]:
parking_translation = {
    'подземная': 'underground',
    'наземная': 'surface',
    'открытая': 'open',
    'многоуровневая': 'multilevel',
    'на крыше': 'on_roof'}

In [94]:
df_tested['parking'] = df_tested['parking'].map(parking_translation)

In [95]:
renovation_translation = {
    'Дизайнерский': 'designer',
    'Евроремонт': 'euro_renovation',
    'Косметический': 'cosmetic',
    'Без ремонта': 'no_renovation'
}

In [96]:
df_tested['renovation'] = df_tested['renovation'].map(renovation_translation)

In [97]:
windows_translation = {
    'Во двор': 'courtyard',
    'На улицу и двор': 'street_and_courtyard',
    'На улицу': 'street'
}

In [98]:
df_tested['windows'] = df_tested['windows'].map(windows_translation)

In [99]:
garbage_chute_translation = {
    'Да': 'Yes',
    'Нет': 'No'
}

In [100]:
df_tested['garbage_chute'] = df_tested['garbage_chute'].map(garbage_chute_translation)

In [101]:
building_type_translation = {
    'Монолитный': 'monolithic',
    'Монолитно-кирпичный': 'monolithic_brick',
    'Неизвестно': 'unknown',
    'Панельный': 'panel',
    'Сталинский': 'stalinist',
    'Кирпичный': 'brick',
    'старый фонд': 'old_fund',
    'Блочный': 'block',
    'Деревянный': 'wooden',
    'Газобетонный блок': 'aerated_concrete_block',
    'Щитовой': 'shield'
}


In [102]:
df_tested['building_type'] = df_tested['building_type'].map(building_type_translation)

Приведем типы данных к int, где это возможно, для ускорения работы модели (предположительно)

In [103]:
df_tested['number_of_rooms'] = df_tested['number_of_rooms'].astype('int')
df_tested['balcony'] = df_tested['balcony'].astype('int')
df_tested['building_floors'] = df_tested['building_floors'].astype('int')
df_tested['loggia'] = df_tested['loggia'].astype('int')
df_tested['combined_bathroom'] = df_tested['combined_bathroom'].astype('int')
df_tested['separate_bathroom'] = df_tested['separate_bathroom'].astype('int')
df_tested['passenger_elevator'] = df_tested['passenger_elevator'].astype('int')
df_tested['freight_elevator'] = df_tested['freight_elevator'].astype('int')
df_tested['allowed_with_children'] = df_tested['allowed_with_children'].astype('int')
df_tested['allowed_with_pets'] = df_tested['allowed_with_pets'].astype('int')

In [104]:
# Делаем дамми кодировку
final_df = pd.get_dummies(df_tested, columns=['parking', 'renovation', 'windows', 'garbage_chute', 'building_type'])

In [105]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23166 entries, 0 to 23198
Data columns (total 40 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    23166 non-null  int64  
 1   number_of_rooms                       23166 non-null  int64  
 2   balcony                               23166 non-null  int64  
 3   ceiling_height_m                      23166 non-null  float64
 4   total_area_m2                         23166 non-null  float64
 5   building_floors                       23166 non-null  int64  
 6   apartment_floor                       23166 non-null  int64  
 7   loggia                                23166 non-null  int64  
 8   combined_bathroom                     23166 non-null  int64  
 9   separate_bathroom                     23166 non-null  int64  
 10  passenger_elevator                    23166 non-null  int64  
 11  freight_elevator    

In [106]:
# final_df.to_csv('/Users/alex_eyfler/ds_bootcamp/rent_real_estate_team_pandas/Alexander/data/final_full_data.csv', index=False)